# Install and Import Libraries

In [114]:
!pip install pdfplumber -q
!pip install sentence-transformers
!apt-get install ocrmypdf -q   #installing OCR library 


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer
from nltk.collocations import *
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vedan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vedan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vedan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vedan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Below Code is to convert Resume Pdf to Text

In [4]:
# input_file_path = "/Users/khedved/VEDANG/Documents/yash.pdf"
# output_file_path = "/Users/khedved/VEDANG/Documents/Vedang_Khedekar_2025.txt"

# with pdfplumber.open(input_file_path) as pdf:
#     text = " ".join(page.extract_text(x_tolerance=2).replace("\n", " ") for page in pdf.pages)

# with open(output_file_path, "w", encoding="utf-8") as file:
#     file.write(text)
# text
current_path='E:/ubuntu/VEDANG_STUDIES/SEM8/BTECH_PROJECT/HELPER/final2/'

# Loading locations and Stop-words

In [117]:
# Locations
f1=open(current_path+'in_districts.txt','r', errors = 'ignore')
text1=f1.read()
locations = word_tokenize(text1.replace("\n", " "))

# Generate stopwords.txt file
stop_words=set(stopwords.words('english'))
with open(current_path+'stopwords.txt',"w") as f:
    for word in stop_words:
            f.write(word+"\n")

# Additional stopwords
f2=open(current_path+'stopwords.txt','r', errors = 'ignore')
text2=f2.read()
stopwords_additional = word_tokenize(text2.replace("\n", " "))
print("locations and stopwords loaded")
# print("locations: ",locations)
# print("stopwords_additional: ",stopwords_additional)

locations and stopwords loaded


# Loading function and apply them to clean resume data

In [118]:
# Below function is used to apply natural language processing on resumes
def nlp(text):
    """Tokenizes, removes stopwords, and lemmatizes the given text."""
    if pd.isna(text):  # Handle NaN values
        return ""
    
    text = text.lower().replace("\n", " ")  # Normalize text
    words = word_tokenize(text)

    # Define stopwords
    stop_words = set(stopwords.words('english') + list(punctuation) + list("●") + 
                     list('–') + list('’') + locations + stopwords_additional)
    
    # Remove stopwords
    words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    processed_text = [lemmatizer.lemmatize(word) for word in words]
    
    return processed_text  # Returning list of words for further processing


In [119]:
# Below function is used to get key words from resumes
def get_key_word(tokens):
    """Finds repeated bigrams (two-word phrases) in tokenized text."""
    finder = BigramCollocationFinder.from_words(tokens)
    key_phrases = []

    for bigram, count in finder.ngram_fd.items():
        if count > 1:  # Only keep phrases appearing more than once
            key_phrases.append(" ".join(bigram))
    
    return key_phrases


In [120]:
# Load cleaned resume dataset
file_path = current_path+"cleaned_file.csv"
df = pd.read_csv(file_path)

# Display first few rows
df.head()


,user_id,resume_id,resume
0,1,RES_001,SUMMARY Certified QA tester with more than 8 y...
1,506,RES_506,SUMMARY Over 7 yearsâ experience in IT indus...
2,518,RES_518,SUMMARY +7 years of extensive experience in So...
3,508,RES_508,Vedang Khedekar khedekarvp21.comp@coeptech.ac....
4,2,RES_002,Sandesh Sahane 7340414303 | sandeshssahane23@g...


In [121]:
# Apply NLP function to the "resume" column
df["Processed_Resume"] = df["resume"].astype(str).apply(nlp)

# Save processed resume data
processed_resume_csv =current_path+ "processed_resume_data.csv"
df.to_csv(processed_resume_csv, index=False)

print(f"Processed resumes saved to: {processed_resume_csv}")

# Display first few rows of processed data
df.head()


Processed resumes saved to: E:/ubuntu/VEDANG_STUDIES/SEM8/BTECH_PROJECT/HELPER/final2/processed_resume_data.csv


,user_id,resume_id,resume,Processed_Resume
0,1,RES_001,SUMMARY Certified QA tester with more than 8 y...,"[summary, certified, qa, tester, 8, year, prov..."
1,506,RES_506,SUMMARY Over 7 yearsâ experience in IT indus...,"[summary, 7, yearsâ, experience, industry, 5..."
2,518,RES_518,SUMMARY +7 years of extensive experience in So...,"[summary, +7, year, extensive, experience, sof..."
3,508,RES_508,Vedang Khedekar khedekarvp21.comp@coeptech.ac....,"[vedang, khedekar, khedekarvp21.comp, coeptech..."
4,2,RES_002,Sandesh Sahane 7340414303 | sandeshssahane23@g...,"[sandesh, sahane, 7340414303, sandeshssahane23..."


In [122]:
# Apply keyword extraction
df["Key_Resume"] = df["Processed_Resume"].apply(get_key_word)

# Save key resume data
key_resume_csv = current_path+"key_resume_data.csv"
df.to_csv(key_resume_csv, index=False)

print(f"Key resume data saved to: {key_resume_csv}")

# Display first few rows with key phrases
df.head()

Key resume data saved to: E:/ubuntu/VEDANG_STUDIES/SEM8/BTECH_PROJECT/HELPER/final2/key_resume_data.csv


,user_id,resume_id,resume,Processed_Resume,Key_Resume
0,1,RES_001,SUMMARY Certified QA tester with more than 8 y...,"[summary, certified, qa, tester, 8, year, prov...","[qa tester, quality assurance, maintain test, ..."
1,506,RES_506,SUMMARY Over 7 yearsâ experience in IT indus...,"[summary, 7, yearsâ, experience, industry, 5...","[sql server, operating system, database backup..."
2,518,RES_518,SUMMARY +7 years of extensive experience in So...,"[summary, +7, year, extensive, experience, sof...","[testing using, soap ui, manual testing, quali..."
3,508,RES_508,Vedang Khedekar khedekarvp21.comp@coeptech.ac....,"[vedang, khedekar, khedekarvp21.comp, coeptech...","[vedang khedekar, khedekar khedekarvp21.comp, ..."
4,2,RES_002,Sandesh Sahane 7340414303 | sandeshssahane23@g...,"[sandesh, sahane, 7340414303, sandeshssahane23...","[web android, android portfolio, pattern techn..."


# Import Job Data Set

In [5]:
#Import scraped data from souce 1
# df = pd.read_csv("E:/ubuntu/VEDANG_STUDIES/SEM8/BTECH_PROJECT/HELPER/india_jobs_dummy.csv")
df = pd.read_csv(current_path+"india_jobs_dummy.csv")

df.head(2)

,JobID,title,company,experience,salary,location,job description,skills,post_date,webpage,company overview,industry,job highlights,All
0,1000,Cybersecurity Analyst,IBM India,5-7 years,INR 600000-1200000 per annum,Chennai,Provide security awareness training. Monitor s...,Secure Coding Risk Assessment Incident Respons...,2025-01-12,http://www.bala.com/,"IBM India offers AI, cloud computing, and ente...",E-commerce,Participate in hackathons and innovation chall...,Cybersecurity Analyst IBM India Chennai Partic...
1,1001,Machine Learning Engineer,Tech Mahindra,7-10 years,INR 2500000-3500000 per annum,NaN,Improve model accuracy with hyperparameter tun...,"Big Data Frameworks (Spark, Hadoop) Supervised...",2025-02-18,http://www.loyal.biz/,Tech Mahindra offers IT services and business ...,Finance,Participate in hackathons and innovation chall...,Machine Learning Engineer Tech Mahindra Parti...


In [6]:
df.shape

(1200, 14)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   JobID             1200 non-null   int64 
 1   title             1200 non-null   object
 2   company           1200 non-null   object
 3   experience        1200 non-null   object
 4   salary            1200 non-null   object
 5   location          951 non-null    object
 6   job description   1200 non-null   object
 7   skills            1200 non-null   object
 8   post_date         1200 non-null   object
 9   webpage           1200 non-null   object
 10  company overview  1200 non-null   object
 11  industry          1200 non-null   object
 12  job highlights    1200 non-null   object
 13  All               1200 non-null   object
dtypes: int64(1), object(13)
memory usage: 131.4+ KB


In [8]:
# Load the dataset and update the All field
file_path = current_path+"india_jobs_dummy.csv"
df = pd.read_csv(file_path)

# Fill NaN values with empty strings to avoid issues while concatenating
df.fillna("", inplace=True)

# Combine required columns into a single "All" column
df["All"] = df["title"] + " " + df["company"] + " " + df["location"] + " " + \
            df["job highlights"] + " " + df["job description"] + " " + df["skills"] +" "+ \
            df["company overview"] + " " + df["industry"] + " " + df["experience"]

# Save the updated DataFrame back to CSV
df.to_csv(file_path, index=False)

print("CSV file updated successfully!")
df.info()



CSV file updated successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   JobID             1200 non-null   int64 
 1   title             1200 non-null   object
 2   company           1200 non-null   object
 3   experience        1200 non-null   object
 4   salary            1200 non-null   object
 5   location          1200 non-null   object
 6   job description   1200 non-null   object
 7   skills            1200 non-null   object
 8   post_date         1200 non-null   object
 9   webpage           1200 non-null   object
 10  company overview  1200 non-null   object
 11  industry          1200 non-null   object
 12  job highlights    1200 non-null   object
 13  All               1200 non-null   object
dtypes: int64(1), object(13)
memory usage: 131.4+ KB


# Testing with scrapped data

In [9]:
# Load the dataset and update the All field
file_path = current_path+"naukri_final_scraping_2.csv"
df2 = pd.read_csv(file_path)

# Fill NaN values with empty strings to avoid issues while concatenating
df2.fillna("", inplace=True)

# Combine required columns into a single "All" column

df2["All"] = df2["title"] + " " + df2["company"] + " " + df2["location"] + " " + \
            df2["job highlights"] + " " + df2["job description"] + " " + df2["skills"] +" " + \
            df2["company overview"] + " " + df2["industry"] + " " + df2["experience"]

# df2=df2.drop(columns=['data source'])
# Save the updated DataFrame back to CSV
df2.to_csv(file_path, index=False)

print("CSV file updated successfully!")
df2.info()


CSV file updated successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   JobID             1249 non-null   int64 
 1   title             1249 non-null   object
 2   company           1249 non-null   object
 3   experience        1249 non-null   object
 4   salary            1249 non-null   object
 5   location          1249 non-null   object
 6   job description   1249 non-null   object
 7   skills            1249 non-null   object
 8   post_date         1249 non-null   object
 9   webpage           1249 non-null   object
 10  company overview  1249 non-null   object
 11  industry          1249 non-null   object
 12  job highlights    1249 non-null   object
 13  All               1249 non-null   object
dtypes: int64(1), object(13)
memory usage: 136.7+ KB


# Merge generated and scrapped Data

In [13]:
df_combined=pd.concat([df,df2],ignore_index=True)
df_combined['index']=df_combined.index
df_combined.to_csv(current_path+"combined.csv",index=False)
df_combined

,JobID,title,company,experience,salary,location,job description,skills,post_date,webpage,company overview,industry,job highlights,All,index
0,1000,Cybersecurity Analyst,IBM India,5-7 years,INR 600000-1200000 per annum,Chennai,Provide security awareness training. Monitor s...,Secure Coding Risk Assessment Incident Respons...,2025-01-12,http://www.bala.com/,"IBM India offers AI, cloud computing, and ente...",E-commerce,Participate in hackathons and innovation chall...,Cybersecurity Analyst IBM India Chennai Partic...,0
1,1001,Machine Learning Engineer,Tech Mahindra,7-10 years,INR 2500000-3500000 per annum,,Improve model accuracy with hyperparameter tun...,"Big Data Frameworks (Spark, Hadoop) Supervised...",2025-02-18,http://www.loyal.biz/,Tech Mahindra offers IT services and business ...,Finance,Participate in hackathons and innovation chall...,Machine Learning Engineer Tech Mahindra Parti...,1
2,1002,Backend Developer,Google India,7-10 years,INR 1200000-1800000 per annum,,Write clean and maintainable backend code. Ens...,Serverless Architecture Containerization (Dock...,2025-01-01,http://www.misra.com/,"Google India focuses on search, cloud computin...",Healthcare,Work closely with product managers to enhance ...,Backend Developer Google India Work closely w...,2
3,1003,Product Manager,Accenture,5-7 years,INR 2500000-3500000 per annum,Pune,Work with stakeholders for strategic planning....,Product Strategy User Experience (UX) Competit...,2025-01-30,https://www.mallick.com/,Accenture is a global professional services co...,Automobile,Competitive salary package with performance-ba...,Product Manager Accenture Pune Competitive sal...,3
4,1004,Software Engineer,SAP Labs India,1-3 years,INR 1200000-1800000 per annum,,Integrate third-party services and APIs. Devel...,Object-Oriented Programming Problem Solving Te...,2025-01-23,https://www.dasgupta-mane.net/,SAP Labs India focuses on enterprise software ...,Finance,Work on cutting-edge technologies in a fast-pa...,Software Engineer SAP Labs India Work on cutt...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,90125504932,Software Developer 2,Oracle,3 - 5 years,Not Disclosed,Bengaluru,Oracle Analytics is a complete platform with r...,"Unix, Automation, Coding, Debugging, Perl, Win...",30+ Days Ago,https://www.naukri.com/job-listings-software-d...,Not Available,Not Available,Not Available,Software Developer 2 Oracle Bengaluru Not Avai...,2444
2445,90125504948,Software Developer 3 - Web Technologies,Oracle,3 - 5 years,Not Disclosed,Bengaluru,Department DescriptionThe worlds most complete...,"Product management, Analytical, Debugging, Jav...",30+ Days Ago,https://www.naukri.com/job-listings-software-d...,Not Available,Not Available,Not Available,Software Developer 3 - Web Technologies Oracle...,2445
2446,120225502409,Embedded Software Developer,Roots Group of Companies,3 - 4 years,Not Disclosed,Coimbatore,1.Proficient in embedded software development ...,"Project development, C++, C, Simulation, ISO, ...",18 Days Ago,https://www.naukri.com/job-listings-embedded-s...,Not Available,Not Available,Not Available,Embedded Software Developer Roots Group of Com...,2446
2447,70225012187,Associate Software Developer,TEKsystems,3 - 6 years,Not Disclosed,Hyderabad,"Mandatory skillsReactJs, HTML, CSS, JavaScript...","ReactJs, NextJS, TypeScript, CSS, JavaScript, ...",23 Days Ago,https://www.naukri.com/job-listings-associate-...,A US-based $4.2 billion renowned brand in IT t...,Not Available,Not Available,Associate Software Developer TEKsystems Hydera...,2447


# Get Recommendation function to get to N jobs

In [129]:
def get_recommendation(top, df, scores):
    recommendation = pd.DataFrame(columns=['JobID', 'title', 'industry', 'job description', 'score'])
    for count, i in enumerate(top):
        recommendation.at[count, 'JobID'] = df.index[i]
        recommendation.at[count, 'title'] = df['title'][i]
        recommendation.at[count, 'industry'] = df['industry'][i]
        recommendation.at[count, 'job description'] = df['job description'][i]
        recommendation.at[count, 'score'] = scores[count]
    return recommendation


# Algorithms to calculate cosine scores

In [130]:
# TFIDF function to calculate cosine simmilarity
def TFIDF(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_jobid = tfidf_vectorizer.fit_transform(scraped_data)
    user_tfidf = tfidf_vectorizer.transform(cv)
    cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x), tfidf_jobid)
    return list(cos_similarity_tfidf)

In [131]:
# Count Vectorization function to calculate cosine simmilarity
def count_vectorize(scraped_data, cv):
    count_vectorizer = CountVectorizer()
    count_jobid = count_vectorizer.fit_transform(scraped_data)
    user_count = count_vectorizer.transform(cv)
    cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x), count_jobid)
    return list(cos_similarity_countv)

In [132]:
# KNN function to calculate top n recommendations
def KNN(scraped_data, cv, n_neighbors=100):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    job_tfidf_matrix = tfidf_vectorizer.fit_transform(scraped_data)  # Fit on job data
    cv_tfidf_matrix = tfidf_vectorizer.transform(cv)  # Transform candidate CV

    # Fit KNN model
    knn_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn_model.fit(job_tfidf_matrix)

    # Find nearest neighbors
    NNs = knn_model.kneighbors(cv_tfidf_matrix, return_distance=True)
    top_indices = NNs[1][0]  # Top job indices
    similarity_scores = 1 - NNs[0][0]  # Convert distance to similarity score (cosine)

    # Get job recommendations
    knn_recommendations = get_recommendation(top_indices, df_combined, similarity_scores)
    return knn_recommendations

In [134]:
# Load data set in resume data frame
resume_df = pd.read_csv(current_path+"key_resume_data.csv")

# TFIDF

In [135]:
# This cell calls TFIDF for each resume and job data set
final_recommendations = pd.DataFrame()
all_recommendations_TFIDF = []

for index, row in resume_df.iterrows():
    user_id = row['user_id']
    resume_text = row['Processed_Resume']
    similarity_scores = TFIDF(df_combined['All'], [resume_text])
    top = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:100]
    list_scores = [similarity_scores[i][0][0] for i in top]
    recommendation = get_recommendation(top, df_combined, list_scores)
    recommendation["user_id"] = user_id
    all_recommendations_TFIDF.append(recommendation)
    # break  # Process only the first resume for debugging
print("TFIDF Calculated")

TFIDF Calculated


In [136]:
# Store and display Recommendations using TFIDF

# final_recommendations = pd.concat(all_recommendations_TFIDF, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
print(all_recommendations_TFIDF)
print("Recommendations saved successfully for TFIDF.")


[   JobID                                          title       industry  \
0   1688            Software Developer Engineer In Test  Not Available   
1   1539                           Software Developer 3  Not Available   
2   1535                           Software Developer 4  Not Available   
3   1748            Software Developer Engineer in Test  Not Available   
4   1516                           Software Developer 3  Not Available   
..   ...                                            ...            ...   
95  2424                           Software Developer 3  Not Available   
96  1964                             Software Developer  Not Available   
97  2302        IBM Public Cloud CTO Software Developer  Not Available   
98  1743  Software Developer Engineer For Testing(SDET)  Not Available   
99  1243                             Software developer  Not Available   

                                      job description     score  user_id  
0   As Full Stack Software Test Aut

# CV

In [137]:
# This cell calls Count Vectorization for each resume and job data set
all_recommendations_CV = []

for index, row in resume_df.iterrows():
    
    user_id = row['user_id']
    resume_text = row['Processed_Resume']
    similarity_scores = count_vectorize(df_combined['All'], [resume_text])
    top = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:100]
    list_scores = [similarity_scores[i][0][0] for i in top]
    recommendation = get_recommendation(top, df_combined, list_scores)
    recommendation["user_id"] = user_id
    all_recommendations_CV.append(recommendation)
    # break  # Process only the first resume for debugging
print("Count Vectorization Calculated")

Count Vectorization Calculated


In [138]:
# Store and display Recommendations using Count Vectorization

# final_recommendations = pd.concat(all_recommendations_CV, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
print(all_recommendations_CV)
print("Recommendations saved successfully for Count Vectorization.")


[   JobID                                    title       industry  \
0   1688      Software Developer Engineer In Test  Not Available   
1   1520  Software Developer Engineer in Test - I  Not Available   
2   1535                     Software Developer 4  Not Available   
3   1539                     Software Developer 3  Not Available   
4   1240           Software Developer Engineer II  Not Available   
..   ...                                      ...            ...   
95  2381             Software Developer (ASP.Net)  Not Available   
96  1523   Senior Software Developer - Automation  Not Available   
97   289                           Data Scientist     Automobile   
98   730                        Software Engineer     Healthcare   
99  1567                       Software Developer  Not Available   

                                      job description     score  user_id  
0   As Full Stack Software Test Automation Enginee...   0.32463        1  
1   ResponsibilitiesManage the e

# KNN

In [139]:
# This cell calls KNN for each resume and job data set
all_recommendations_KNN = []

for index, row in resume_df.iterrows():
    user_id = row['user_id']
    resume_text = row['Processed_Resume']
    recommendation = KNN(df_combined['All'], [resume_text])
    recommendation["user_id"] = user_id
    all_recommendations_KNN.append(recommendation)
    # break  # Process only the first resume for debugging
print("KNN Calculated")

KNN Calculated


In [140]:
# Store and display Recommendations using KNN

# final_recommendations = pd.concat(all_recommendations_KNN, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
print(all_recommendations_KNN)
print("Recommendations saved successfully for KNN.")


[   JobID                                          title       industry  \
0   1688            Software Developer Engineer In Test  Not Available   
1   1539                           Software Developer 3  Not Available   
2   1535                           Software Developer 4  Not Available   
3   1748            Software Developer Engineer in Test  Not Available   
4   1516                           Software Developer 3  Not Available   
..   ...                                            ...            ...   
95  2424                           Software Developer 3  Not Available   
96  1964                             Software Developer  Not Available   
97  2302        IBM Public Cloud CTO Software Developer  Not Available   
98  1743  Software Developer Engineer For Testing(SDET)  Not Available   
99  1243                             Software developer  Not Available   

                                      job description     score  user_id  
0   As Full Stack Software Test Aut

# Display TFIDF, CV and KNN result together

In [141]:

# Convert lists of recommendations into DataFrames
df_TFIDF = pd.concat(all_recommendations_TFIDF, ignore_index=True)
df_CV = pd.concat(all_recommendations_CV, ignore_index=True)
df_KNN = pd.concat(all_recommendations_KNN, ignore_index=True)
# df_BERT = pd.concat(all_recommendations_BERT, ignore_index=True)

# Select only relevant columns before merging
df_TFIDF = df_TFIDF[['user_id', 'JobID', 'score']].rename(columns={'score': 'score_TFIDF'})
df_CV = df_CV[['user_id', 'JobID', 'score']].rename(columns={'score': 'score_CV'})
df_KNN = df_KNN[['user_id', 'JobID', 'score']].rename(columns={'score': 'score_KNN'})
# df_BERT = df_BERT[['user_id', 'JobID', 'score']].rename(columns={'score': 'score_BERT'})

# Merge based on common 'user_id' and 'job_id'
merged_df = df_TFIDF.merge(df_CV, on=["user_id", "JobID"], how="inner")
merged_df = merged_df.merge(df_KNN, on=["user_id", "JobID"], how="inner")
# merged_df = merged_df.merge(df_BERT, on=["user_id", "JobID"], how="inner")
# merged_df = df_CV.merge(df_BERT, on=["user_id", "JobID"], how="inner")

# Add BERT scores separately using LEFT JOIN
# merged_df = merged_df.merge(df_BERT, on=["user_id", "JobID"], how="left")

# Sort results for each user based on the highest TF-IDF score (or any preferred metric)
# merged_df = merged_df.sort_values(by=["user_id", "score_TFIDF"], ascending=[True, False])

# Display the final merged recommendations
print(merged_df.head())

# Save to CSV if needed
merged_df.to_csv("final_recommendations.csv", index=False)

   user_id JobID score_TFIDF  score_CV score_KNN
0        1  1688    0.292058   0.32463  0.292058
1        1  1539    0.239966  0.243797  0.239966
2        1  1535    0.239777  0.243887  0.239777
3        1  1748     0.23718  0.229168   0.23718
4        1  1516    0.219485  0.202462  0.219485


# BERT


In [143]:
# Load Pretrained Sentence Transformer Model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')
def BERT(scraped_data, cv, top_n=200):
    # Encode Job Descriptions
    job_embeddings = bert_model.encode(scraped_data, convert_to_tensor=True)

    # Encode Candidate's CV
    cv_embedding = bert_model.encode(cv, convert_to_tensor=True)

    # Compute Cosine Similarity
    similarities = np.dot(job_embeddings, cv_embedding.T).flatten()

    # Get Top Matches
    top_indices = similarities.argsort()[-top_n:][::-1]  # Top 200 jobs

    # Get Recommendations
    bert_recommendations = get_recommendation_bert(top_indices, df_combined, similarities[top_indices])

    return bert_recommendations

def get_recommendation_bert(top, df, scores):
    recommendation = pd.DataFrame(columns=['JobID', 'title', 'industry', 'job description', 'score'])
    for count, i in enumerate(top):
        recommendation.at[count, 'JobID'] = df.iloc[i]['JobID']  # Fix: Use iloc to get the correct JobID
        recommendation.at[count, 'title'] = df.iloc[i]['title']
        recommendation.at[count, 'industry'] = df.iloc[i]['industry']
        recommendation.at[count, 'job description'] = df.iloc[i]['job description']
        recommendation.at[count, 'score'] = scores[count]
    return recommendation


In [144]:
all_recommendations_BERT = []
for index, row in merged_df.iterrows():
    user_id = row['user_id']
    job_id = row['JobID']

    # Get Job Description for the given JobID
    job_text = df_combined[df_combined.index == job_id]['All'].values

    # Get Resume for the User
    resume_text = resume_df[resume_df['user_id'] == user_id]['Processed_Resume'].values

    if len(job_text) > 0 and len(resume_text) > 0:
        recommendation = BERT(job_text, [resume_text[0]], top_n=1)  # Apply BERT for only this job-user pair
        recommendation["user_id"] = user_id
        recommendation["JobID"] = job_id
        all_recommendations_BERT.append(recommendation)

print("BERT Applied to Merged Jobs and Users")


BERT Applied to Merged Jobs and Users


In [145]:
df_BERT = pd.concat(all_recommendations_BERT, ignore_index=True)
df_BERT = df_BERT[['user_id', 'JobID', 'score']].rename(columns={'score': 'score_BERT'})
# print(df_BERT.head())
# Merge BERT scores with the existing merged DataFrame
final_df = merged_df.merge(df_BERT, on=["user_id", "JobID"], how="left")

# Save Final Results
final_df.to_csv("final_recommendations_with_BERT.csv", index=False)

# Display Final DataFrame
print(final_df.head())


   user_id JobID score_TFIDF  score_CV score_KNN score_BERT
0        1  1688    0.292058   0.32463  0.292058   0.511246
1        1  1539    0.239966  0.243797  0.239966     0.2902
2        1  1535    0.239777  0.243887  0.239777   0.287746
3        1  1748     0.23718  0.229168   0.23718   0.453017
4        1  1516    0.219485  0.202462  0.219485   0.342026


# Scale and assign weights

In [148]:
# Scale it
from sklearn.preprocessing import MinMaxScaler
# slr = MinMaxScaler()
# final[["KNN", "TF-IDF", 'CV']] = slr.fit_transform(final[["KNN", "TF-IDF", 'CV']])
# print(final['KNN'])
# Multiply by weights
a=final_df['score_KNN']/4
b=final_df['score_TFIDF']/4
c=final_df['score_CV']/8
d=final_df['score_BERT']/2
# print(a)
# print(b)
# print(c)

# final['KNN'] = a
# final['TF-IDF'] = b
# final['CV'] = c
final_df['Final'] = a+b+c+d

final_df = final_df.sort_values(by=["user_id", "Final"], ascending=[True, False])

final_df.to_csv("final.csv", index=False)
print(final_df)
# final.sort_values(by="Final", ascending=False)

     user_id JobID score_TFIDF  score_CV score_KNN score_BERT     Final
0          1  1688    0.292058   0.32463  0.292058   0.511246   0.44223
12         1  2350    0.185272  0.220928  0.185272   0.544656   0.39258
3          1  1748     0.23718  0.229168   0.23718   0.453017  0.373744
19         1  2444    0.177338  0.179962  0.177338   0.486068  0.354198
11         1  1915    0.189236  0.181256  0.189236    0.46727   0.35091
..       ...   ...         ...       ...       ...        ...       ...
166      518   280      0.1471  0.222948    0.1471   0.321553  0.262195
146      518  1378    0.185381  0.171158  0.185381   0.275674  0.251922
180      518   799    0.134055  0.197446  0.134055   0.315896  0.249656
169      518   239    0.145643  0.187183  0.145643   0.289819  0.241129
182      518  1732    0.132097  0.194162  0.132097   0.267861  0.224249

[382 rows x 7 columns]


# Return Top 10 jobs for each user

In [150]:
# Group by user_id and get the top 10 recommendations for each user
top_10 = final_df.groupby("user_id").head(10)

# Sort the final result by user_id and Final score
top_10 = top_10.sort_values(by=["user_id", "Final"], ascending=[True, False])

# Display the result
print(top_10)

# Save to CSV
# top_10.to_csv("top_10_recommendations.csv", index=False)


     user_id JobID score_TFIDF  score_CV score_KNN score_BERT     Final
0          1  1688    0.292058   0.32463  0.292058   0.511246   0.44223
12         1  2350    0.185272  0.220928  0.185272   0.544656   0.39258
3          1  1748     0.23718  0.229168   0.23718   0.453017  0.373744
19         1  2444    0.177338  0.179962  0.177338   0.486068  0.354198
11         1  1915    0.189236  0.181256  0.189236    0.46727   0.35091
..       ...   ...         ...       ...       ...        ...       ...
134      518  2350    0.213844  0.226284  0.213844   0.464835  0.367625
133      518  1602     0.21674  0.235913   0.21674   0.457831  0.366775
156      518  2394    0.162361  0.204889  0.162361   0.515309  0.364446
129      518  1516     0.23505  0.217447   0.23505   0.438051  0.363731
138      518  1855    0.196853  0.168761  0.196853    0.46285  0.350947

[80 rows x 7 columns]


In [151]:
# Ensure JobID in top_10 is of the same type as df_combined index
top_10["JobID"] = top_10["JobID"].astype(str)

# Convert index of df_combined to string and reset index
df_combined.index = df_combined.index.astype(str)

# Merge top_10 with df_combined using index as JobID
top_10_with_details = top_10.merge(df_combined, left_on="JobID", right_index=True, how="left")

# Display the result
print(top_10_with_details)

# Save to CSV if needed
top_10_with_details.to_csv("top_10_recommendations_with_details.csv", index=False)


    JobID  user_id JobID_x score_TFIDF  score_CV score_KNN score_BERT  \
0    1688        1    1688    0.292058   0.32463  0.292058   0.511246   
12   2350        1    2350    0.185272  0.220928  0.185272   0.544656   
3    1748        1    1748     0.23718  0.229168   0.23718   0.453017   
19   2444        1    2444    0.177338  0.179962  0.177338   0.486068   
11   1915        1    1915    0.189236  0.181256  0.189236    0.46727   
..    ...      ...     ...         ...       ...       ...        ...   
134  2350      518    2350    0.213844  0.226284  0.213844   0.464835   
133  1602      518    1602     0.21674  0.235913   0.21674   0.457831   
156  2394      518    2394    0.162361  0.204889  0.162361   0.515309   
129  1516      518    1516     0.23505  0.217447   0.23505   0.438051   
138  1855      518    1855    0.196853  0.168761  0.196853    0.46285   

        Final       JobID_y  \
0     0.44223  270225500763   
12    0.39258  190225501627   
3    0.373744  171123500897   

# Algorithms for recommending Resumes to Recruiters

In [168]:
def get_recommendation_rev(top, df, scores):
    # Create a list of dictionaries for better performance
    recommendation = pd.DataFrame(columns=['JobID','user_id', 'resume_id','score'])
    
    for count, i in enumerate(top):
        recommendation.at[count, 'user_id'] = df.iloc[i]['user_id']
        recommendation.at[count, 'resume_id'] = df.iloc[i]['resume_id']  # Optional, if available
        recommendation.at[count, 'score'] = scores[count]
    
    return recommendation

In [174]:
def KNN_rev(resume_data, job_desc, n_neighbors=8):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # Fit on all resumes (instead of jobs)
    resume_tfidf_matrix = tfidf_vectorizer.fit_transform(resume_data)
    
    # Transform the given job description using the same vectorizer
    job_tfidf_matrix = tfidf_vectorizer.transform(job_desc)

    # Fit KNN model on resumes
    knn_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn_model.fit(resume_tfidf_matrix)

    # Find nearest resumes
    NNs = knn_model.kneighbors(job_tfidf_matrix, return_distance=True)
    top_indices = NNs[1][0]  # Top resume indices
    similarity_scores = 1 - NNs[0][0]  # Convert distance to similarity score (cosine similarity)

    # Get resume recommendations
    knn_recommendations = get_recommendation_rev(top_indices, resume_df, similarity_scores)
    return knn_recommendations


# TFIDF for Recruiter

In [169]:
# This cell calls TFIDF for each job to recommend resumes
final_recommendations_rev = pd.DataFrame()
all_recommendations_TFIDF_rev = []

for index, row in df_combined.iterrows():  
    job_id = index  # Assuming job ID is the index
    job_text = row['All']
    
    similarity_scores = TFIDF(resume_df['Processed_Resume'], [job_text])  # Now comparing against resumes
    top = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:100]
    
    list_scores = [similarity_scores[i][0][0] for i in top]
    recommendation = get_recommendation_rev(top, resume_df, list_scores)
    
    recommendation["JobID"] = job_id
    all_recommendations_TFIDF_rev.append(recommendation)

print("TFIDF Calculated for Jobs")




TFIDF Calculated for Jobs


In [170]:
# Store and display Recommendations using TFIDF

# final_recommendations = pd.concat(all_recommendations_TFIDF, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
print(all_recommendations_TFIDF_rev)
print("Recommendations saved successfully for TFIDF.")


[  JobID user_id resume_id     score
0     0     508   RES_508  0.148862
1     0       3   RES_003  0.111992
2     0     518   RES_518  0.110678
3     0       1   RES_001  0.100188
4     0     506   RES_506  0.075726
5     0       2   RES_002  0.044591
6     0       4   RES_004  0.011068
7     0       5   RES_005       0.0,   JobID user_id resume_id     score
0     1     508   RES_508  0.150726
1     1       2   RES_002  0.064037
2     1     506   RES_506  0.061853
3     1       3   RES_003  0.061179
4     1     518   RES_518  0.060078
5     1       1   RES_001  0.055214
6     1       5   RES_005  0.042547
7     1       4   RES_004  0.040411,   JobID user_id resume_id     score
0     2     508   RES_508    0.2097
1     2       3   RES_003  0.198871
2     2       2   RES_002  0.129502
3     2     506   RES_506  0.069051
4     2     518   RES_518  0.063616
5     2       1   RES_001  0.054237
6     2       4   RES_004  0.016208
7     2       5   RES_005  0.004626,   JobID user_id resume_i

# CV for Recruiter

In [171]:
# This cell calls Count Vectorization for each job to recommend resumes
all_recommendations_CV_rev = []

for index, row in df_combined.iterrows():
    job_id = index
    job_text = row['All']
    
    similarity_scores = count_vectorize(resume_df['Processed_Resume'], [job_text])
    top = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:100]
    
    list_scores = [similarity_scores[i][0][0] for i in top]
    recommendation = get_recommendation_rev(top, resume_df, list_scores)
    
    recommendation["JobID"] = job_id
    all_recommendations_CV_rev.append(recommendation)

print("Count Vectorization Calculated for Jobs")


Count Vectorization Calculated for Jobs


In [172]:
# Store and display Recommendations using CV

# final_recommendations = pd.concat(all_recommendations_TFIDF, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
print(all_recommendations_CV_rev)
print("Recommendations saved successfully for TFIDF.")


[  JobID user_id resume_id     score
0     0     508   RES_508  0.163922
1     0       3   RES_003  0.141993
2     0     518   RES_518   0.12961
3     0       1   RES_001  0.118825
4     0     506   RES_506  0.102931
5     0       2   RES_002  0.066515
6     0       4   RES_004  0.054465
7     0       5   RES_005  0.013865,   JobID user_id resume_id     score
0     1     508   RES_508  0.198986
1     1       2   RES_002  0.106474
2     1       3   RES_003  0.098678
3     1       4   RES_004  0.082429
4     1     506   RES_506  0.077447
5     1       5   RES_005  0.072132
6     1     518   RES_518  0.070796
7     1       1   RES_001  0.061882,   JobID user_id resume_id     score
0     2     508   RES_508  0.220737
1     2       3   RES_003  0.207793
2     2       2   RES_002   0.16019
3     2     506   RES_506  0.113405
4     2     518   RES_518  0.097713
5     2       1   RES_001  0.087646
6     2       4   RES_004  0.060507
7     2       5   RES_005  0.014003,   JobID user_id resume_i

# KNN for Recruiter

In [175]:

# This cell calls KNN for each job to recommend resumes
all_recommendations_KNN_rev = []

for index, row in df_combined.iterrows():
    job_id = index
    job_text = row['All']
    
    recommendation = KNN_rev(resume_df['Processed_Resume'], [job_text])
    
    recommendation["JobID"] = job_id
    all_recommendations_KNN_rev.append(recommendation)

print("KNN Calculated for Jobs")


KNN Calculated for Jobs


In [176]:
# Store and display Recommendations using KNN

# final_recommendations = pd.concat(all_recommendations_TFIDF, ignore_index=True)
# final_recommendations.to_csv("user_recommendations_tfidf.csv", index=False)
print(all_recommendations_KNN_rev)
print("Recommendations saved successfully for TFIDF.")


[  JobID user_id resume_id     score
0     0     508   RES_508  0.148862
1     0       3   RES_003  0.111992
2     0     518   RES_518  0.110678
3     0       1   RES_001  0.100188
4     0     506   RES_506  0.075726
5     0       2   RES_002  0.044591
6     0       4   RES_004  0.011068
7     0       5   RES_005       0.0,   JobID user_id resume_id     score
0     1     508   RES_508  0.150726
1     1       2   RES_002  0.064037
2     1     506   RES_506  0.061853
3     1       3   RES_003  0.061179
4     1     518   RES_518  0.060078
5     1       1   RES_001  0.055214
6     1       5   RES_005  0.042547
7     1       4   RES_004  0.040411,   JobID user_id resume_id     score
0     2     508   RES_508    0.2097
1     2       3   RES_003  0.198871
2     2       2   RES_002  0.129502
3     2     506   RES_506  0.069051
4     2     518   RES_518  0.063616
5     2       1   RES_001  0.054237
6     2       4   RES_004  0.016208
7     2       5   RES_005  0.004626,   JobID user_id resume_i

# Scale and Assign Weights for Recruiter

In [180]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Convert lists of recommendations into DataFrames
df_TFIDF_rev = pd.concat(all_recommendations_TFIDF_rev, ignore_index=True)
df_CV_rev = pd.concat(all_recommendations_CV_rev, ignore_index=True)
df_KNN_rev = pd.concat(all_recommendations_KNN_rev, ignore_index=True)

# Select only relevant columns before merging
df_TFIDF_rev = df_TFIDF_rev[['JobID', 'user_id', 'score']].rename(columns={'score': 'score_TFIDF'})
df_CV_rev = df_CV_rev[['JobID', 'user_id', 'score']].rename(columns={'score': 'score_CV'})
df_KNN_rev = df_KNN_rev[['JobID', 'user_id', 'score']].rename(columns={'score': 'score_KNN'})

# Merge on 'JobID' and 'user_id' to align resume recommendations for each job
merged_df_rev = df_TFIDF_rev.merge(df_CV_rev, on=["JobID", "user_id"], how="inner")
merged_df_rev = merged_df_rev.merge(df_KNN_rev, on=["JobID", "user_id"], how="inner")

# Scale scores using MinMaxScaler to normalize across methods
scaler = MinMaxScaler()
merged_df_rev[['score_KNN', 'score_TFIDF', 'score_CV']] = scaler.fit_transform(
    merged_df_rev[['score_KNN', 'score_TFIDF', 'score_CV']]
)

# Calculate final weighted score (equal weightage to all)
merged_df_rev['Final_Score'] = (merged_df_rev['score_KNN'] + merged_df_rev['score_TFIDF'] + merged_df_rev['score_CV']) / 3

# Sort results for each job by Final_Score
final_df_rev = merged_df_rev.sort_values(by=["JobID", "Final_Score"], ascending=[True, False])

# Save final recommendations to CSV
final_df_rev.to_csv("final_resume_recommendations.csv", index=False)

# Display top recommendations
print(final_df_rev.head())


  JobID user_id  score_TFIDF  score_CV  score_KNN  Final_Score
0     0     508     0.266280  0.314190   0.266280     0.282250
1     0       3     0.200329  0.272160   0.200329     0.224272
2     0     518     0.197978  0.248424   0.197978     0.214793
3     0       1     0.179213  0.227753   0.179213     0.195393
4     0     506     0.135457  0.197288   0.135457     0.156068


# Top 3 Recommended Resume

In [186]:
# Group by user_id and get top 3 recommendations per user
top_3 = final_df_rev.groupby("JobID").head(3)

# Sort the final result by user_id and Final Score
top_3 = top_3.sort_values(by=["JobID", "Final_Score"], ascending=[True, False])

# Save the final top recommendations to CSV
top_3.to_csv("final_resume_recommendations.csv", index=False)

# Display the result
print(top_3)

     JobID user_id  score_TFIDF  score_CV  score_KNN  Final_Score
0        0     508     0.266280  0.314190   0.266280     0.282250
1        0       3     0.200329  0.272160   0.200329     0.224272
2        0     518     0.197978  0.248424   0.197978     0.214793
8        1     508     0.269614  0.381399   0.269614     0.306876
9        1       2     0.114547  0.204080   0.114547     0.144391
...    ...     ...          ...       ...        ...          ...
7985   998       1     0.369268  0.435215   0.369268     0.391250
7986   998     508     0.320650  0.375711   0.320650     0.339004
7992   999     508     0.294571  0.367929   0.294571     0.319024
7993   999     518     0.220782  0.257017   0.220782     0.232861
7994   999       3     0.200774  0.224693   0.200774     0.208747

[7347 rows x 6 columns]


# Load Interaction Data Set

In [53]:
file_path = current_path+"final.csv"
df_interaction = pd.read_csv(file_path)

# Generate random actual_application values (0 or 1) based on a probability distribution
np.random.seed(42)  # For reproducibility
df_interaction["actual_application"] = np.random.choice([0, 1,2], size=len(df), p=[0.3, 0.3,0.4])  

# Save the updated dataset
output_file = current_path+"interaction_final.csv"
df_interaction.to_csv(output_file, index=False)
output_file

'E:/ubuntu/VEDANG_STUDIES/SEM8/BTECH_PROJECT/HELPER/final2/interaction_final.csv'

# Define Evaluation Functions

In [68]:
import numpy as np
from sklearn.metrics import average_precision_score, roc_auc_score


def precision_at_k(df, k=3, threshold=1):
    """
    Calculates Precision@K treating items with relevance >= threshold as relevant.
    Threshold:
        1 → Counts "Saved (1)" and "Applied (2)" as relevant.
        2 → Counts only "Applied (2)" as relevant.
    """
    precision_scores = []
    
    for user, user_df in df.groupby("user_id"):
        top_k = user_df.nlargest(k, "Final")  # Select top K jobs
        relevant_items = (top_k["actual_application"] >= threshold).sum()  # Relevant items at threshold
        precision = relevant_items / k  # Relevant / Total
        precision_scores.append(precision)
    
    return np.mean(precision_scores)

def recall_at_k(df, k=3, threshold=1):
    """
    Calculates Recall@K treating items with relevance >= threshold as relevant.
    """
    recall_scores = []
    
    for user, user_df in df.groupby("user_id"):
        top_k = user_df.nlargest(k, "Final")  # Select top K jobs
        relevant_jobs = (user_df["actual_application"] >= threshold).sum()  # Total relevant jobs
        recall = top_k["actual_application"].apply(lambda x: 1 if x >= threshold else 0).sum() / relevant_jobs if relevant_jobs > 0 else 0
        recall_scores.append(recall)
    
    return np.mean(recall_scores)


def mean_average_precision(df):
    """
    Computes Mean Average Precision (MAP) treating 'Saved (1)' and 'Applied (2)' as relevant.
    """
    map_scores = []
    
    for user, user_df in df.groupby("user_id"):
        y_true = (user_df["actual_application"] > 0).astype(int)  # Convert to binary (0=irrelevant, 1=relevant)
        y_scores = user_df["Final"].values  # Predicted scores
        
        if y_true.sum() > 0:  # Only compute if at least one relevant item exists
            map_scores.append(average_precision_score(y_true, y_scores))
    
    return np.mean(map_scores)

def ndcg_at_k(df, k=3):
    """
    Computes NDCG@K using multi-level relevance.
    """
    def dcg_at_k(relevances, k):
        """
        Computes Discounted Cumulative Gain (DCG).
        """
        relevances = np.asfarray(relevances)[:k]
        return np.sum(relevances / np.log2(np.arange(2, relevances.size + 2)))

    def idcg_at_k(relevances, k):
        """
        Computes Ideal DCG (IDCG).
        """
        ideal_relevances = sorted(relevances, reverse=True)
        return dcg_at_k(ideal_relevances, k)

    ndcg_scores = []
    
    for user, user_df in df.groupby("user_id"):
        top_k = user_df.nlargest(k, "Final")["actual_application"].values
        idcg = idcg_at_k(top_k, k)
        dcg = dcg_at_k(top_k, k)
        ndcg_scores.append(dcg / idcg if idcg > 0 else 0)
    
    return np.mean(ndcg_scores)

def auc_roc(df):
    """
    Computes AUC-ROC treating 'Saved (1)' and 'Applied (2)' as positive cases.
    """
    binary_labels = df["actual_application"].apply(lambda x: 1 if x >= 1 else 0)
    return roc_auc_score(binary_labels, df["Final"])


In [69]:
print("Precision@3 (Saved + Applied):", precision_at_k(df_interaction, k=3, threshold=1))
print("Precision@3 (Applied Only):", precision_at_k(df_interaction, k=3, threshold=2))
print("Recall@3 (Saved + Applied):", recall_at_k(df_interaction, k=3, threshold=1))
print("Recall@3 (Applied Only):", recall_at_k(df_interaction, k=3, threshold=2))
# print("MAP:", mean_average_precision(df_interaction))
print("NDCG@3:", ndcg_at_k(df_interaction, k=3))
print("AUC-ROC (Saved+Applied vs Not Clicked):", auc_roc(df_interaction))



Precision@3 (Saved + Applied): 0.7916666666666666
Precision@3 (Applied Only): 0.375
Recall@3 (Saved + Applied): 0.07973091995657919
Recall@3 (Applied Only): 0.06045203506503816
NDCG@3: 0.8780445991986852
AUC-ROC (Saved+Applied vs Not Clicked): 0.474118718021157
